In [5]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [6]:
with open("D:\\React\\neplearn\\Data\\KUexam_questions.json","r",encoding="utf-8") as f:
    data = json.load(f)
    

In [7]:
questions = data["questions"]

In [8]:
embeddings= np.array([q["embedding"] for q in questions])

In [9]:
norms = np.linalg.norm(embeddings, axis=1, keepdims= True)

In [10]:
embeddings_normalized = embeddings/ norms

In [11]:
def random_centroids(embeddings, k):
    indices = np.random.choice(len(embeddings), size=k, replace=False)
    return embeddings[indices]

In [12]:
centroids = random_centroids(embeddings, 20)

In [13]:
centroids

array([[-0.03841667,  0.09902942, -0.08539046, ..., -0.05056182,
         0.03279319, -0.01171589],
       [-0.01015604,  0.01884941, -0.02778734, ...,  0.08134744,
         0.13778727, -0.04774292],
       [ 0.06810799,  0.03946166, -0.02364482, ...,  0.0156238 ,
         0.01191835, -0.00539442],
       ...,
       [ 0.00513949,  0.02512921, -0.09050431, ...,  0.0478999 ,
         0.04252009, -0.02535645],
       [-0.04352959,  0.03221422, -0.06604343, ...,  0.08941482,
         0.04340914,  0.00613072],
       [-0.07905579,  0.04101683, -0.04981722, ...,  0.09798084,
         0.0632806 ,  0.00560518]], shape=(20, 384))

In [14]:
def get_labels(embeddings, centroids):
    distances = np.linalg.norm(embeddings[:, np.newaxis, :] - centroids[np.newaxis, :, :],axis=2) 
    labels = np.argmin(distances, axis=1)
    return labels

In [15]:
labels = get_labels(embeddings, centroids)


In [16]:
def update_centroids(embeddings, labels, k):
    d = embeddings.shape[1]
    new_centroids = np.zeros((k, d))

    for i in range(k):
        cluster_points = embeddings[labels == i]

        if len(cluster_points) == 0:
            # reinitialize empty cluster
            new_centroids[i] = embeddings[np.random.randint(len(embeddings))]
        else:
            new_centroids[i] = cluster_points.mean(axis=0)

    return new_centroids


In [17]:
max_iterations = 100
k = 15
tolerance = 1e-6

centroids = random_centroids(embeddings_normalized, k)
iteration = 0

while iteration < max_iterations:
    old_centroids = centroids.copy()

    labels = get_labels(embeddings_normalized, centroids)
    centroids = update_centroids(embeddings_normalized, labels, k)

    # Re-normalize centroids
    centroids /= np.linalg.norm(centroids, axis=1, keepdims=True)

    # Convergence check
    if np.allclose(centroids, old_centroids, atol=tolerance):
        print(f"Converged at iteration {iteration}")
        break

    iteration += 1


Converged at iteration 6


In [18]:
for q, cluster_id in zip(questions, labels):
    q["cluster_id"] = int(cluster_id)


In [19]:
cluster_id_tocheck = 0

cluster_0_questions = [
    q for q in questions if q["cluster_id"] == cluster_id_tocheck
]


In [20]:
for i,q in enumerate(cluster_0_questions[:125]):
    print(f"{i+1}. ({q['year']}) {q['cleaned_text']}")

1. (2014) summarize the rules for naming identifiers are uppercase letters equivalent to lowercase letters
2. (2014) what is the purpose of the keyword void where is this keyword used
3. (2014) what restrictions are applied using keywaords
4. (2014) can the names of formal arguments within a function coincide with the names of other variables defined outside of the function explain
5. (2016) with example explain the two types of repetition control
6. (2017) how does variable differ from a keyword
7. (2018) describe general programming rules
8. (2018) mention the significance of functions
9. (2022) illustrate the use of auto and static keywords with examples
10. (2022) what are arguments explain their importance with examples
11. (2022) discuss a menudriven program with an example
12. (2024) what is an identifier
13. (2024) explain `break` and `continue` statements with examples
